In [2]:
import pickle

In [3]:
code_path = '/mnt/yuxuan/Code_summarization/csn_dataset/original/code_tokens.pkl'
comment_path = '/mnt/yuxuan/Code_summarization/csn_dataset/original/docstring_tokens.pkl'

In [4]:
code = pickle.load(open(code_path,'rb'))

In [5]:
comment = pickle.load(open(comment_path,'rb'))

In [6]:
code['train'][0]

['bind', 'indexed', 'elements', 'to', 'the', 'supplied', 'collection']

In [7]:
comment['train'][0]

['bind', 'indexed', 'elements', 'to', 'the', 'supplied', 'collection']

In [8]:
code_path2 = '/mnt/yuxuan/Code_summarization/csn_dataset/original/code_tokens_javalang.pkl'

In [9]:
code2 = pickle.load(open(code_path2,'rb'))

In [10]:
code2['train'][0]

['protected',
 'final',
 'void',
 'bindIndexed',
 '(',
 'ConfigurationPropertyName',
 'name',
 ',',
 'Bindable',
 '<',
 '?',
 '>',
 'target',
 ',',
 'AggregateElementBinder',
 'elementBinder',
 ',',
 'ResolvableType',
 'aggregateType',
 ',',
 'ResolvableType',
 'elementType',
 ',',
 'IndexedCollectionSupplier',
 'result',
 ')',
 '{',
 'for',
 '(',
 'ConfigurationPropertySource',
 'source',
 ':',
 'getContext',
 '(',
 ')',
 '.',
 'getSources',
 '(',
 ')',
 ')',
 '{',
 'bindIndexed',
 '(',
 'source',
 ',',
 'name',
 ',',
 'target',
 ',',
 'elementBinder',
 ',',
 'result',
 ',',
 'aggregateType',
 ',',
 'elementType',
 ')',
 ';',
 'if',
 '(',
 'result',
 '.',
 'wasSupplied',
 '(',
 ')',
 '&&',
 'result',
 '.',
 'get',
 '(',
 ')',
 '!=',
 'null',
 ')',
 '{',
 'return',
 ';',
 '}',
 '}',
 '}']

In [4]:
data = '/mnt/yuxuan/Code_summarization0323/csn_dataset/train_new2/processed_100.train.pt2'

In [5]:
import torch

In [6]:
d = torch.load(data)

In [17]:
tree = d['train_xe']['trees']

In [16]:
def get_data_trees(trees):
    data_trees = []
    for t_json in trees:
        for k, node in t_json.items():
            # if 'parent' not in node:
            #     root_idx ='#'
            #     break
            if node['parent'] == None:
                root_idx = k
        # if root_idx == "#":
        #     continue
        tree = json2tree_binary(t_json, Tree(), root_idx)
        data_trees.append(tree)

    return data_trees 

def json2tree_binary(tree_json, tree, idx, prev=None):
    if prev == None:
        tree.parent = prev
        tree.idx = idx
    children = [c for c in tree_json[idx]['children'] if c.startswith(Constants.NODE_FIX)]
    if len(children) == 2:
        for c in children:
            t = Tree()
            t.parent = prev
            t.idx = c
            t.num_children = 0
            k = json2tree_binary(tree_json, t, c, tree)
            tree.add_child(k)
    else:
        tree.parent = prev
        tree.num_children = 0
        # print "tree_json[idx]['children']: "
        # print tree_json[idx]['children']
        tree.content = tree_json[idx]['children'][0]
    return tree

In [17]:
class Tree(object):
    def __init__(self):
        self.parent = None
        self.num_children = 0
        self.children = list()
        self.leaf_states = list()
        self._leaf_count = 0
        self._leaf_contents = []
        self.content = None

    def add_child(self,child):
        child.parent = self
        self.num_children += 1
        self.children.append(child)

    def size(self):
        # if getattr(self,'_size'):
        #     return self._size
        count = 1
        for i in xrange(self.num_children):
            count += self.children[i].size()
        self._size = count
        return self._size

    def depth(self):
        # if getattr(self,'_depth'):
        #     return self._depth
        count = 0
        if self.num_children>0:
            for i in xrange(self.num_children):
                child_depth = self.children[i].depth()
                if child_depth>count:
                    count = child_depth
            count += 1
        self._depth = count
        return self._depth

    def leaf_count_(self):
        # count = 0
        if not self.children:
            self._leaf_count = 1
        else:
            for c in self.children:
                leaf_count = c.leaf_count()
                self._leaf_count += leaf_count
                # print "self._leaf_count: ", self._leaf_count
        # self._leaf_count = count
        return self._leaf_count

    def leaf_count(self):
        count = 0
        # if not self.children:
        #     self._leaf_count = 1
        # else:
        # if not self.children:
        #     count += 1
        if self.children:
            for c in self.children:
                leaf_count = c.leaf_count()
                count += leaf_count
                # if leaf_count>count:
                #     count = leaf_count

        else:
            count += 1
        # self._leaf_count = count
        return count
                # self._leaf_count += leaf_count
                # print "self._leaf_count: ", self._leaf_count
        # self._leaf_count = count
        # return self._leaf_count

    def leaf_contents(self):
        if not self.children:
            # print "self.content: "
            # print self.content
            self._leaf_contents.append(self.content)
        else:
            for c in self.children:
                leaf_content = c.leaf_contents()
                self._leaf_contents.extend(leaf_content)

        return self._leaf_contents

In [28]:
from lib.data import Constants
new_tree = get_data_trees(tree)

In [7]:
data2 = '/mnt/yuxuan/Code_summarization0323/csn_dataset/fix_bug/processed_100.train.pt'

In [8]:
d2 = torch.load(data2)

In [9]:
data3 = '/mnt/yuxuan/Code_summarization0323/csn_dataset/fix_bug/processed_100.train.pt2'

In [43]:
d2.keys()

dict_keys(['train_xe', 'test', 'train_pg', 'valid', 'dicts'])

In [10]:
d2['train_xe'].keys()

dict_keys(['src', 'trees', 'tgt'])

In [57]:
t1 = d['train_xe']['tgt'][0]

In [58]:
t2 =d2['train_xe']['tgt'][0]

In [60]:
t1.equal(t2)

True

In [61]:
key = ['train_xe', 'test', 'train_pg', 'valid']

i = 0
j = 0
for k in key:
    while j < len(d2[k]['tgt']):
        if d[k]['tgt'][i].equal(d2[k]['tgt'][j]):
            d[k]['src'][i] = d2[k]['src'][j]
            i += 1
            j += 1
        else:
            j += 1
            
                   

In [46]:
d['train_xe']['src'][0]

tensor([   92,    26,    32,     1,     4,     1,    63,     8,     1,    13,
           46,    15,   214,     8,     1,     1,     8,  9280,     1,     8,
         9280,  2868,     8,     1,    43,     5,    10,    37,     4,     1,
          199,    28,   857,     4,     5,     6, 15933,     4,     5,     5,
           10,     1,     4,   199,     8,    63,     8,   214,     8,     1,
            8,    43,     8,     1,     8,  2868,     5,     7,    12,     4,
           43,     6,     1,     4,     5,    39,    43,     6,    40,     4,
            5,    30,    18,     5,    10,    14,     7,     9,     9,     9])

In [47]:
d['train_xe']['tgt'][0]

tensor([ 4,  5,  6,  7,  8,  9, 10,  3])

In [48]:
torch.save(d,'/mnt/yuxuan/Code_summarization0323/csn_dataset/fix_bug/processed_100.train.pt2' )

In [15]:
len(d['train_xe']['tgt'])

430584

In [14]:
len(d['train_xe']['trees'])

430584

In [11]:
len(d['train_xe']['src'])

430584

In [18]:
tree = d['train_xe']['trees']

In [19]:
from lib.data import Constants
new_tree = get_data_trees(tree)

In [20]:
new_tree[0]

In [22]:
d['train_xe']['trees'][64*200:64*201]

[{'1*NODEFIX6': {'node': 'VariableDeclarator',
   'children': ['1*NODEFIX8', '1*NODEFIX17'],
   'parent': '1*NODEFIX4'},
  '1*NODEFIX0': {'node': 'MethodDeclaration',
   'children': ['1*NODEFIX1', '1*NODEFIX18'],
   'parent': None},
  '1*NODEFIX5': {'node': 'ReferenceType',
   'children': ['ReferenceType'],
   'parent': '1*NODEFIX4'},
  '1*NODEFIX1': {'node': 'BasicType',
   'children': ['BasicType'],
   'parent': '1*NODEFIX0'},
  '1*NODEFIX8': {'node': '.', 'children': ['.'], 'parent': '1*NODEFIX6'},
  '1*NODEFIX9': {'node': 'MethodInvocation',
   'children': ['STR_'],
   'parent': '1*NODEFIX17'},
  '1*NODEFIX18': {'node': 'Tmp',
   'children': ['1*NODEFIX2', '1*NODEFIX19'],
   'parent': '1*NODEFIX0'},
  '1*NODEFIX19': {'node': 'Tmp',
   'children': ['1*NODEFIX4', '1*NODEFIX13'],
   'parent': '1*NODEFIX18'},
  '1*NODEFIX4': {'node': 'val',
   'children': ['1*NODEFIX5', '1*NODEFIX6'],
   'parent': '1*NODEFIX19'},
  '1*NODEFIX17': {'node': 'Tmp',
   'children': ['1*NODEFIX9', '1*NODEFIX

In [26]:
d['valid']['trees'][64*15]

{'1*NODEFIX34': {'node': 'Tmp',
  'children': ['1*NODEFIX22', '1*NODEFIX36'],
  'parent': '1*NODEFIX31'},
 '1*NODEFIX35': {'node': 'Tmp',
  'children': ['1*NODEFIX15', '1*NODEFIX16'],
  'parent': '1*NODEFIX32'},
 '1*NODEFIX36': {'node': 'Tmp',
  'children': ['1*NODEFIX23', '1*NODEFIX25'],
  'parent': '1*NODEFIX34'},
 '1*NODEFIX37': {'node': 'Tmp',
  'children': ['1*NODEFIX6', '1*NODEFIX38'],
  'parent': '1*NODEFIX33'},
 '1*NODEFIX30': {'node': 'Tmp',
  'children': ['1*NODEFIX3', '1*NODEFIX33'],
  'parent': '1*NODEFIX0'},
 '1*NODEFIX31': {'node': 'Tmp',
  'children': ['1*NODEFIX20', '1*NODEFIX34'],
  'parent': '1*NODEFIX16'},
 '1*NODEFIX32': {'node': 'Tmp',
  'children': ['1*NODEFIX14', '1*NODEFIX35'],
  'parent': '1*NODEFIX10'},
 '1*NODEFIX33': {'node': 'Tmp',
  'children': ['1*NODEFIX4', '1*NODEFIX37'],
  'parent': '1*NODEFIX30'},
 '1*NODEFIX38': {'node': 'Tmp',
  'children': ['1*NODEFIX10', '1*NODEFIX27'],
  'parent': '1*NODEFIX37'},
 '1*NODEFIX25': {'node': 'ClassReference',
  'chil

In [37]:
tree = d['valid']['trees'][64*15:64*16]
new_tree = get_data_trees(tree)

In [33]:
tree_lengths = []
for tree in new_tree:
    l_c = tree.leaf_count()
    tree_lengths.append(l_c)

In [34]:
tree_lengths

[13,
 12,
 13,
 8,
 6,
 7,
 11,
 5,
 5,
 5,
 13,
 11,
 5,
 8,
 6,
 9,
 10,
 5,
 10,
 17,
 11,
 7,
 7,
 7,
 9,
 7,
 6,
 7,
 7,
 8,
 8,
 4,
 7,
 7,
 8,
 21,
 4,
 9,
 7,
 7,
 5,
 6,
 11,
 6,
 10,
 14,
 19,
 6,
 8,
 9,
 37,
 7,
 16,
 7,
 9,
 7,
 9,
 7,
 13,
 5,
 8,
 1,
 1,
 6]

In [39]:
len(tree[-2])

1